In [1]:
from osgeo import gdal

import numpy as np

import h5py
import random
from scipy import ndarray
from skimage import exposure

from random import random as rd
import cv2 as cv

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.losses import binary_crossentropy as bce
from keras.losses import categorical_crossentropy as cce
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping #,TensorBoard#, ReduceLROnPlateau, LearningRateScheduler
from keras.utils.np_utils import to_categorical  
import utils as utils
import predict as predict

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
K.clear_session()

In [4]:
def image_generator(image_path,label_path,image_id,tile_id,y_coord,x_coord, batch,isAugment=False):
    
    def raster2arr(raster, bnd,x,y,wndo):
        rr = np.zeros((wndo, wndo, bnd))
        newValue = 0
        for b in range(bnd):
            band = raster.GetRasterBand(b+1)
            img = band.ReadAsArray(x, y, wndo, wndo)
            noDataValue = band.GetNoDataValue()
            img[img == noDataValue] = newValue
            rr[:,:,b] = img
        return rr.astype(np.uint8)
    
    def com_gen(image,label,isAugment):
        K.set_image_data_format('channels_last')
        def combine_generator(gen1, gen2):
            while True:
                yield(gen1.next(), gen2.next())
        
        if isAugment is True:
            data_gen_args = dict(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=45,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
#                              brightness_range=[0.8,1.2],
                             zoom_range=0.2,
                             fill_mode='nearest',
                             data_format = "channels_last",
                             rescale = 1./255)
        else:
            data_gen_args = dict(data_format = "channels_last",
                                 rescale = 1./255)
        
        image_datagen = ImageDataGenerator(**data_gen_args)
        mask_datagen = ImageDataGenerator(**data_gen_args)
        image_datagen.fit(image, augment=isAugment, rounds=1, seed=42)
        mask_datagen.fit(label, augment=isAugment, rounds=1, seed=42)
        image_generator = image_datagen.flow(image, batch_size=1, seed=42)
        mask_generator = mask_datagen.flow(label, batch_size=1, seed=42)
        
        return combine_generator(image_generator,mask_generator)

    
    total_images = len(y_coord)
    all_images_id =list(range(total_images))
    random.seed(948)
    random.shuffle(all_images_id)
    
    current_image_id = 0
    while (current_image_id<total_images):
        
        random.shuffle(all_images_id)
        
        batch_input = np.zeros((batch,size,size,3))
        batch_output = np.zeros((batch,size,size,1))
        
       
        for batch_index in range(batch):
            index = all_images_id[current_image_id]
            x = x_coord[index]
            y = y_coord[index]
            img_id = image_id[index]
            
        
            raster = gdal.Open(image_path[img_id])
            batch_input[batch_index,:,:,:] = raster2arr(raster,3,x,y,size)
            raster = None
            
            raster = gdal.Open(label_path[img_id])
            batch_output[batch_index,:,:,:]= raster2arr(raster,1,x,y,size)*255
            raster = None  
            
        current_image_id += 1
        return (com_gen(batch_input, batch_output,isAugment))

def dev_test_generator(image_path,label_path,image_id,tile_id,y_coord,x_coord, batch):    
    
    def raster2arr(raster, bnd,x,y,wndo):
        rr = np.zeros((wndo, wndo, bnd))
        newValue = 0
        for b in range(bnd):
            band = raster.GetRasterBand(b+1)
            img = band.ReadAsArray(x, y, wndo, wndo)
            noDataValue = band.GetNoDataValue()
            img[img == noDataValue] = newValue
            rr[:,:,b] = img
        return rr.astype(np.uint8)

    def com_gen(image2,label2):
        K.set_image_data_format('channels_last')
        def combine_generator(gen1, gen2):
            while True:
                yield(gen1.next(), gen2.next())                                   
  
        data_gen_args2 = dict(data_format = "channels_last",
                             rescale = 1./255)
        image_datagen2 = ImageDataGenerator(**data_gen_args2)
        mask_datagen2 = ImageDataGenerator(**data_gen_args2)
        #image_datagen.fit(image, augment=True, rounds=5, seed=42)
        #mask_datagen.fit(label, augment=True, rounds=5, seed=42)
        image_generator2 = image_datagen2.flow(image2, batch_size=1, seed=56)
        mask_generator2 = mask_datagen2.flow(label2, batch_size=1, seed=56)
        val_generator = combine_generator(image_generator2,mask_generator2)
        return val_generator
    
    total_images = len(y_coord)
    all_images_id =list(range(total_images))
    random.seed(948)
    random.shuffle(all_images_id)
    
    current_image_id = 0
    while (current_image_id<total_images):
        
        random.shuffle(all_images_id)
        
        batch_input = np.zeros((batch,size,size,3))
        batch_output = np.zeros((batch,size,size,1))
        
       
        for batch_index in range(batch):
            index = all_images_id[current_image_id]
            x = x_coord[index]
            y = y_coord[index]
            img_id = image_id[index]
            
        
            raster = gdal.Open(image_path[img_id])
            batch_input[batch_index,:,:,:] = raster2arr(raster,3,x,y,size)
            raster = None
            
            raster = gdal.Open(label_path[img_id])
            batch_output[batch_index,:,:,:]= raster2arr(raster,1,x,y,size)*255
            raster = None  
            
        current_image_id += 1
        return (com_gen(batch_input, batch_output))


In [5]:
size = 256
def unet(size, lri, input_height = size, input_width = size, nClasses = 1):

    K.set_image_data_format('channels_last')
    
    input_size = (input_width, input_height, 3)
    input1 = Input(shape = input_size)
    n = 64
    drate1 = 0.5
    drate2 = 0.5
    drate3 = 0.5
    drate4 = 0.5
    drate5 = 0.5
    lmbd = 0.0001
    #bn01 = ( BatchNormalization())(input1) 
    
    #drop0 = Dropout(0)(input1)
    conv1 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(input1)
    drop1 = Dropout(drate1)(conv1)
    bn11 = ( BatchNormalization())(drop1) 

    conv1 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(bn11)
    drop12 = Dropout(drate2)(conv1)
    bn12 = ( BatchNormalization())(drop12) 

    pool1 = MaxPooling2D(pool_size=(2, 2))(bn12)
    
    conv2 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',  data_format= "channels_last", kernel_initializer = 'he_normal')(pool1)
    drop2 = Dropout(drate1)(conv2) 
    bn21 = ( BatchNormalization())(drop2)

    conv2 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',  data_format= "channels_last", kernel_initializer = 'he_normal')(bn21)
    drop22 = Dropout(drate2)(conv2)  
    bn22 = ( BatchNormalization())(drop22)   
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn22)
    
    conv3 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(pool2)
    drop3 = Dropout(drate1)(conv3)
    bn31 = ( BatchNormalization())(drop3)

    conv3 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn31)
    drop32 = Dropout(drate2)(conv3)
    bn32 = ( BatchNormalization())(drop32) 
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn32)
    
    conv4 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(pool3)
    drop4 = Dropout(drate1)(conv4)
    bn41 = ( BatchNormalization())(drop4) 

    conv4 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn41)
    drop42 = Dropout(drate5)(conv4)
    bn42 = ( BatchNormalization())(drop42) 
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn42)
   
    conv5 = Conv2D(n*16, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(pool4)
    drop5 = Dropout(drate5)(conv5)
    bn51 = ( BatchNormalization())(drop5) 
    conv5 = Conv2D(n*16, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn51)
    drop52 = Dropout(drate4)(conv5)
    bn52 = ( BatchNormalization())(drop52) 
    
    up6 = Conv2D(n*8, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',data_format= "channels_last",  kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn52))
    merge6 = concatenate([bn42,up6], axis = 3)
    #merge6 = concatenate([conv4,up6], axis = 3)
    
    conv6 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge6)
    drop6 = Dropout(drate3)(conv6)
    bn61 = ( BatchNormalization())(drop6) 

    conv6 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn61)
    drop62 = Dropout(drate4)(conv6)
    bn62 = ( BatchNormalization())(drop62) 
    
    up7 = Conv2D(n*4, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn62))
    
    merge7 = concatenate([bn32,up7], axis = 3)
    
    
    conv7 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge7)
    drop7 = Dropout(drate3)(conv7)
    bn71 = ( BatchNormalization())(drop7) 

    conv7 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn71)
    drop72 = Dropout(drate4)(conv7)
    bn72 = ( BatchNormalization())(drop72) 
    

    up8 = Conv2D(n*2, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn72))
    
    
    merge8 = concatenate([bn22,up8], axis = 3)
    
    conv8 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge8)
    drop8 = Dropout(drate3)(conv8)
    bn81 = ( BatchNormalization())(drop8) 

    conv8 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn81)
    drop82 = Dropout(drate4)(conv8)
    bn82 = ( BatchNormalization())(drop82) 
    
    up9 = Conv2D(n, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn82))
    merge9 = concatenate([bn12,up9], axis = 3)
    
    conv9 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge9)
    drop9 = Dropout(drate3)(conv9)
    bn91 = ( BatchNormalization())(drop9) 

    conv9 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn91)
    drop92 = Dropout(drate4)(conv9)
    bn92 = ( BatchNormalization())(drop92) 
    
    conv10 = Conv2D(nClasses, (1,1), kernel_regularizer=l2(lmbd), activation = 'sigmoid', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn92)
    model = Model(inputs = input1, outputs = conv10)
    
    model.compile(optimizer = Adam(lr  = lri), loss= utils.dice_loss, metrics = [utils.iou, utils.dice])
   
    return model


In [6]:
filepath1 = './images'
filepath2 = './labels_old'

size = 256
#List of all jps and tif files
ipath = ((utils.get_all_images(filepath1, 'jpg')))
cpath = ((utils.get_all_images(filepath2, 'tif')))

assert len(ipath) == len(cpath)

ipath_train, ipath_val, cpath_train, cpath_val = train_test_split(ipath,cpath, train_size = 0.6, random_state=64, shuffle = True)
ipath_dev, ipath_test, cpath_dev, cpath_test = train_test_split(ipath_val,cpath_val, train_size = 0.5, random_state=128, shuffle = True)

In [7]:
def fetch_tiles_info(ipath):
    img_id  = []
    tile_id = []
    y_coord = []
    x_coord = []


    for i in range(len(ipath)):
        img_id,tile_id,y_coord,x_coord = utils.apply_fetch_all_tiles(ipath[i],img_id,tile_id,y_coord,x_coord,1,i)
        
#         img_id,tile_id,y_coord,x_coord = utils.apply_fetch_tiles_at_random(ipath[i],img_id,tile_id,y_coord,x_coord,len(img_id),len(img_id),i)
    
    return img_id,tile_id,y_coord,x_coord

In [8]:
img_id_train,tile_id_train,y_coord_train,x_coord_train = fetch_tiles_info(ipath_train)
img_id_train,tile_id_train,y_coord_train,x_coord_train = utils.shuffle_list(img_id_train,tile_id_train,y_coord_train,x_coord_train)

img_id_dev,tile_id_dev,y_coord_dev,x_coord_dev = fetch_tiles_info(ipath_dev)
img_id_dev,tile_id_dev,y_coord_dev,x_coord_dev = utils.shuffle_list(img_id_dev,tile_id_dev,y_coord_dev,x_coord_dev)

img_id_test,tile_id_test,y_coord_test,x_coord_test = fetch_tiles_info(ipath_test)
img_id_test,tile_id_test,y_coord_test,x_coord_test = utils.shuffle_list(img_id_test,tile_id_test,y_coord_test,x_coord_test)

image: 0, number of tiles: 1
image: 1, number of tiles: 1
image: 2, number of tiles: 1
image: 3, number of tiles: 1
image: 4, number of tiles: 1
image: 5, number of tiles: 1
image: 6, number of tiles: 1
image: 7, number of tiles: 1
image: 8, number of tiles: 1
image: 9, number of tiles: 1
image: 10, number of tiles: 1
image: 11, number of tiles: 1
image: 12, number of tiles: 1
image: 13, number of tiles: 1
image: 14, number of tiles: 1
image: 15, number of tiles: 1
image: 16, number of tiles: 1
image: 17, number of tiles: 1
image: 18, number of tiles: 1
image: 19, number of tiles: 1
image: 20, number of tiles: 1
image: 21, number of tiles: 1
image: 22, number of tiles: 1
image: 23, number of tiles: 1
image: 24, number of tiles: 1
image: 25, number of tiles: 1
image: 26, number of tiles: 1
image: 27, number of tiles: 1
image: 28, number of tiles: 1
image: 29, number of tiles: 1
image: 30, number of tiles: 1
image: 31, number of tiles: 1
image: 0, number of tiles: 1
image: 1, number of t

In [9]:
batch = 8

train_generator = image_generator(ipath_train,cpath_train,img_id_train,tile_id_train,y_coord_train,x_coord_train, batch)

val_generator = image_generator(ipath_dev,cpath_dev,img_id_dev,tile_id_dev,y_coord_dev,x_coord_dev, 1)


In [11]:
def training(size):
    K.clear_session()
             
    K.set_image_data_format('channels_last')
    seed = 23
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    lri = 1e-3

    model = unet(size, lri)
    model.summary()
    
    checkpoint = ModelCheckpoint("./model_1.h5", monitor='val_dice', verbose=1, save_best_only=True, save_weights_only=False, mode='max', peroid=1)
    
    early = EarlyStopping(monitor='val_dice', min_delta=0, patience=20, verbose=1, mode='auto')
    history = model.fit(train_generator, steps_per_epoch = (len(y_coord_train)//batch), epochs = 50 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(y_coord_dev)))    
    

training(size)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         dropout[0][0]                    
______________________________________________________________________________________________

Epoch 1/50

Epoch 00001: val_dice improved from -inf to 0.34804, saving model to ./model_1.h5
4/4 - 26s - loss: 2.0519 - iou: 0.1790 - dice: 0.3033 - val_loss: 1.9841 - val_iou: 0.2107 - val_dice: 0.3480
Epoch 2/50

Epoch 00002: val_dice improved from 0.34804 to 0.35167, saving model to ./model_1.h5
4/4 - 14s - loss: 1.9148 - iou: 0.2554 - dice: 0.4063 - val_loss: 1.9530 - val_iou: 0.2134 - val_dice: 0.3517
Epoch 3/50

Epoch 00003: val_dice improved from 0.35167 to 0.38800, saving model to ./model_1.h5
4/4 - 14s - loss: 1.7753 - iou: 0.3524 - dice: 0.5208 - val_loss: 1.8939 - val_iou: 0.2407 - val_dice: 0.3880
Epoch 4/50

Epoch 00004: val_dice did not improve from 0.38800
4/4 - 12s - loss: 1.6770 - iou: 0.4248 - dice: 0.5961 - val_loss: 1.8856 - val_iou: 0.2284 - val_dice: 0.3719
Epoch 5/50

Epoch 00005: val_dice did not improve from 0.38800
4/4 - 12s - loss: 1.6064 - iou: 0.4715 - dice: 0.6407 - val_loss: 1.8797 - val_iou: 0.2115 - val_dice: 0.3491
Epoch 6/50

Epoch 00006: val_dice di

In [12]:
K.clear_session()

model = load_model('./model_1.h5', custom_objects={'dice_loss': utils.dice_loss,'iou': utils.iou, 'dice': utils.dice})

print('Metrics: ', model.metrics_names)  
l,i,d = model.evaluate(train_generator, verbose=1, sample_weight=None, steps=len(x_coord_train)//4)
print('train: ',l,i,d)

l,i,d = model.evaluate(val_generator, verbose=1, sample_weight=None, steps=len(x_coord_dev))
print('val: ',l,i,d)

l,i,d = model.evaluate(dev_test_generator(ipath_test,cpath_test,img_id_test,tile_id_test,y_coord_test,x_coord_test, 1), batch_size = 1, verbose=1, sample_weight=None, steps=len(x_coord_test))
print('test: ',l,i,d)

   
K.clear_session()
del model 


W0606 19:02:44.043192 109064 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0606 19:02:44.043192 109064 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0606 19:02:44.043192 109064 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instruct

Metrics:  ['loss', 'iou', 'dice']
8/8 [==============================] - 4s 490ms/step - loss: 0.8417 - iou: 0.4394 - dice: 0.6105
train:  0.8416581153869629 0.4394036 0.6105198
11/11 [==============================] - 5s 421ms/step - loss: 0.8372 - iou: 0.4441 - dice: 0.6150
val:  0.8371680974960327 0.4440691 0.6150098
11/11 [==============================] - 5s 422ms/step - loss: 0.7846 - iou: 0.5011 - dice: 0.6676
test:  0.7845747470855713 0.5010604 0.6676031


In [13]:
folder = "./images"
mpath = "./model_1.h5"
flist = utils.get_all_images(folder, "jpg")
predict.predict_all_images(flist,mpath,4)

predicting
(256, 256)
predicting
predicting
predicting
0: 65426, 1: 110
0: 65426, 1: 110
image 1 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 65536, 1: 0
0: 65536, 1: 0
image 2 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 65347, 1: 189
0: 65347, 1: 189
image 3 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 51533, 1: 14003
0: 51533, 1: 14003
image 4 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 56348, 1: 9188
0: 56348, 1: 9188
image 5 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 54834, 1: 10702
0: 54834, 1: 10702
image 6 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 58045, 1: 7491
0: 58045, 1: 7491
image 7 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 51436, 1: 14100
0: 51436, 1: 14100
image 8 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 65536, 1: 0
0: 65536, 1: 0
image 9 saved
predicting
(256, 256)
predicting
predicting
predicting
0: 65536, 